In [5]:
# !/usr/bin/python
# -*- coding: UTF-8 -*-
# word dict

import numpy as np

In [6]:
from collections import Counter

class WordCounter2Dict():
    '''
    vocab as keys, freq as value, dict
    '''
    def __init__(self):
        super().__init__()
        self.word_dict = None
                 
    def get_counter(self, counter, MAX_VOCAB_SIZE):
        self.max = MAX_VOCAB_SIZE - 1 if MAX_VOCAB_SIZE is not "all" else None
        self.word_dict = dict(counter.most_common(self.max))
#         for not common vocab marked as '<unk>', freq = 0
#         self.word_dict['<unk>'] = sum(list(counter.values())) - sum(list(self.word_dict.values()))
        self.word_dict['<unk>'] = 0
    def __str__(self):
        return "{}".format(self.word_dict)
#     id = enumerate in word word_dict.keys()
    def getlen(self):
        return len(self.word_dict)
    def id2word(self):
        return list(self.word_dict.keys())
    def word2id(self):
        return {word: wordid for wordid, word in enumerate(list(self.word_dict))}
    def word_counts(self):
        return np.array(list(self.word_dict.values()), dtype = np.float32)
    def normalized(self):
        return self.word_counts() / sum(self.word_counts())

In [7]:
import torch
from torch.utils import data
from scipy import stats

class WordDataset(data.Dataset):
    def __init__(self, word_list, word_dict, WINDOW, INTERVAL, NEGATIVE):
        super().__init__()
        self.word_list = word_list
        self.word2id = word_dict.word2id()
        self.word_counts = torch.from_numpy(word_dict.word_counts())
        self.normalized = torch.from_numpy(word_dict.normalized())
        self.word_weight = torch.from_numpy(np.array([1-weight for weight in self.word_counts]))
#         zscore tfreq dict
#         self.word_zscore_tfreq = torch.from_numpy(stats.zscore(self.word_counts))
        
#         word list by id according to all text but not sentences
        self.encoded = [self.word2id.get(word, self.word2id['<unk>']) for word in self.word_list]
        self.encoded = torch.Tensor(self.encoded).long()
    
        self.window = WINDOW
        self.negative = NEGATIVE
        self.interval = INTERVAL
    def __len__(self):
        return len(self.word_list)
    
    def __getitem__(self, index):
        '''
        dataset magic function
        
        index: word index in word list
        
        return including:
        central vocab
        context vocabs
        negative samples
        '''   
#         central vocab id
        cen_vocab_id = self.encoded[index]

# ------window types

#         1. interval window
#         former = [index - (t * self.interval) for t in range(1, self.window+1)]
#         former.reverse()
#         latter = [index + (t * self.interval) for t in range(1, self.window+1)]
#         window_index = former + latter
#         2. normal window
        window_index = list(range(index - self.window, index)) + list(range(index + 1, index + self.window + 1))
        index_con_vocab = [i % len(self.word_list) for i in window_index] # circle index
        
#         context vocabs id
        con_vocabs_id = self.encoded[index_con_vocab]
# ------weight
        con_vocabs_weight = self.word_weight[con_vocabs_id]

# ------sampling types
# and in model.py
        
#         1. Negative sampling in (self.word_tfreq), for (NEGATIVE * con_vocabs_id.shape[0]) times
#         (con_vocabs_id.shape): LIST of context vocabs， pick up self.negative * con_vocabs_id.shape[0] words
        neg_samples_id = torch.multinomial(self.normalized, self.negative * con_vocabs_id.shape[0], True)       
#         2. no negative sampling
#         neg_samples_id = 0
    
        return cen_vocab_id, con_vocabs_id, con_vocabs_weight, neg_samples_id

In [8]:
# MAX_VOCAB_SIZE = "all" # the vocabulary size 词汇表多大
# WINDOW = 3 # context window
# INTERVAL = 2
# NEGATIVE = 20 # number of negative samples
# BATCH_SIZE = 32
# # test

# word_dict = WordCounter2Dict()
# word_list = []
# with open("source/test.txt", 'r', encoding='utf-8') as fin:
#     c = Counter()
#     for line in fin.readlines():
#         text = line.strip().split(" ") #分割成词列表
#         word_list.extend(text)
#         c += Counter(text)
#     word_dict.get_counter(c, MAX_VOCAB_SIZE)
    
# word_dataset = WordDataset(word_list, word_dict, WINDOW, INTERVAL, NEGATIVE)
# dataloader = data.DataLoader(word_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# for i, (cen_vocab_id, con_vocabs, con_vocabs_weight, neg_samples) in enumerate(dataloader):
#     if i == 0:
#         print(cen_vocab_id.size(), con_vocabs.size(), con_vocabs_weight.size(), neg_samples.size())
#         break;

torch.Size([32]) torch.Size([32, 6]) torch.Size([32, 6]) torch.Size([32, 120])


= torch.Size([32]) torch.Size([32, 1]) torch.Size([32, 6]) torch.Size([32, 600])